In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
X, y = df.iloc[:,:-1], df["target"]

In [4]:
np.random.seed(42)

In [5]:
y.value_counts()

0    2661
1     539
Name: target, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [7]:
df.isna().sum().sum()

0

In [8]:
newdata = pd.concat([X_train, y_train], axis=1)

In [9]:
pos_X = newdata[newdata.iloc[:,-1] == 1]
neg_X = newdata[newdata.iloc[:,-1] == 0]
print(pos_X.shape, neg_X.shape)
from sklearn.utils import resample, shuffle

pos_upsample = resample(pos_X,
             replace=True,
             n_samples=2129)
print(pos_upsample.shape)
df_res = shuffle(np.concatenate([neg_X, pos_upsample]))
X_train, y_train = df_res[:,:-1], df_res[:, -1]

(431, 806) (2129, 806)
(2129, 806)


In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_tr_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, roc_auc_score
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
roc_auc_score(y_test, y_pred)

/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


0.5208159287106655

In [12]:
from sklearn.model_selection import GridSearchCV

In [20]:
from sklearn.linear_model import LogisticRegression
parameters = {'C':[0.1, 0.5, 1, 2], 'l1_ratio':[0, 0.5, 1]}
svc = LogisticRegression(penalty="elasticnet", solver='saga')
clf = GridSearchCV(svc, parameters,scoring='roc_auc')
clf.fit(X_tr_sc, y_train)
clf.best_score_

/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.7520130874541181

In [21]:
clf.best_params_

{'C': 2, 'l1_ratio': 0.5}

In [22]:
X_tr_sc.shape

(4258, 805)

In [23]:
be = clf.best_estimator_

In [24]:
y_pred = be.predict(X_test_sc)

In [25]:
(y_test == y_pred).mean()

0.675

In [26]:
roc_auc_score(y_test, y_pred)

0.5941938178780284

In [27]:
from sklearn.neighbors import KNeighborsClassifier
parameters = {'n_neighbors':[1, 3, 5], 'weights':['uniform', 'distance'], 'algorithm': ['kd_tree', 'ball_tree']}
svc = KNeighborsClassifier()
clf = GridSearchCV(svc, parameters,scoring='roc_auc')
clf.fit(X_tr_sc, y_train)
clf.best_score_

KeyboardInterrupt: 

In [171]:
clf.best_params_

{'algorithm': 'kd_tree', 'n_neighbors': 5, 'weights': 'distance'}

In [172]:
y_pred = clf.predict(X_test_sc)
roc_auc_score(y_test, y_pred)

0.5443817878028405

In [ ]:
from sklearn.svm import SVC
parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, 1, 5]}
svc = SVC()
clf = GridSearchCV(svc, parameters,scoring='roc_auc')
clf.fit(X_tr_sc, y_train)
clf.best_score_

In [ ]:
y_pred = clf.predict(X_test_sc)
f1_score(y_test, y_pred)

In [54]:
clf.best_params_

{'C': 0.1, 'kernel': 'rbf'}

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
random_grid = { 
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,6,10],
    'criterion' :['gini', 'entropy'],
    'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)] + ['balanced']
}


svc = RandomForestClassifier(n_estimators=100)
clf = GridSearchCV(svc, random_grid,scoring='f1')
clf.fit(X_train, y_train)
clf.best_score_

0.3685728335295786

In [29]:
clf.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'sqrt'}

In [31]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.37726098191214474

In [46]:
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', criterion='gini', max_depth=6,max_features='sqrt')
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=6,
                       max_features='sqrt')

In [19]:
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
f_i = list(zip(np.arange(805),rf.feature_importances_))
f_i.sort(key = lambda x : x[1])
plt.barh([x[0] for x in f_i],[x[1] for x in f_i])

plt.show()

In [41]:
X_train.shape

(2560, 805)

In [47]:
mask = rf.feature_importances_ > 0.0001

In [48]:
mask.shape

(805,)

In [50]:
X_el = X_train.iloc[:, mask]
X_el.shape

(2560, 564)

In [51]:
rf.fit(X_el, y_train)
y_pred = rf.predict(X_test.iloc[:, mask])
f1_score(y_test, y_pred)

0.3722084367245657

In [13]:
from sklearn.ensemble import AdaBoostClassifier

random_grid = { 
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
}


svc = AdaBoostClassifier(n_estimators=100)
clf = GridSearchCV(svc, random_grid,scoring='roc_auc')
clf.fit(X_train, y_train)
clf.best_score_

0.8140934079113362

In [14]:
clf.best_params_

{'learning_rate': 0.1}

In [15]:
y_pred = clf.predict(X_test)
roc_auc_score(y_test, y_pred)

/Users/mariamitrankova/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


0.6301865775549986

In [17]:
import catboost as cb

In [18]:
model = cb.CatBoostClassifier(loss_function='Logloss',  
        iterations=1000,
        auto_class_weights='Balanced'
                              
    )
random_grid = { 
    'learning_rate': [0.01, 0.05, 0.1],
    'depth':[4,6,8],
    'min_data_in_leaf':[1, 3]
}
clf = GridSearchCV(model, random_grid,scoring='roc_auc', cv=3)
clf.fit(X_train, y_train)
clf.best_score_


0:	learn: 0.6917373	total: 96.5ms	remaining: 1m 36s
1:	learn: 0.6898472	total: 115ms	remaining: 57.6s
2:	learn: 0.6874389	total: 135ms	remaining: 44.8s
3:	learn: 0.6853118	total: 153ms	remaining: 38s
4:	learn: 0.6837976	total: 173ms	remaining: 34.4s
5:	learn: 0.6824067	total: 193ms	remaining: 31.9s
6:	learn: 0.6805784	total: 211ms	remaining: 30s
7:	learn: 0.6792518	total: 230ms	remaining: 28.5s
8:	learn: 0.6780803	total: 248ms	remaining: 27.3s
9:	learn: 0.6768002	total: 268ms	remaining: 26.6s
10:	learn: 0.6748702	total: 288ms	remaining: 25.9s
11:	learn: 0.6736235	total: 309ms	remaining: 25.4s
12:	learn: 0.6722008	total: 332ms	remaining: 25.2s
13:	learn: 0.6705878	total: 355ms	remaining: 25s
14:	learn: 0.6698498	total: 378ms	remaining: 24.8s
15:	learn: 0.6688784	total: 398ms	remaining: 24.5s
16:	learn: 0.6682862	total: 419ms	remaining: 24.2s
17:	learn: 0.6669968	total: 437ms	remaining: 23.8s
18:	learn: 0.6652606	total: 458ms	remaining: 23.6s
19:	learn: 0.6636026	total: 476ms	remaining: 

161:	learn: 0.5797510	total: 3.23s	remaining: 16.7s
162:	learn: 0.5792276	total: 3.25s	remaining: 16.7s
163:	learn: 0.5789416	total: 3.27s	remaining: 16.7s
164:	learn: 0.5785422	total: 3.29s	remaining: 16.7s
165:	learn: 0.5781033	total: 3.31s	remaining: 16.6s
166:	learn: 0.5775799	total: 3.33s	remaining: 16.6s
167:	learn: 0.5770924	total: 3.35s	remaining: 16.6s
168:	learn: 0.5766594	total: 3.37s	remaining: 16.6s
169:	learn: 0.5762052	total: 3.39s	remaining: 16.5s
170:	learn: 0.5758269	total: 3.41s	remaining: 16.5s
171:	learn: 0.5754279	total: 3.43s	remaining: 16.5s
172:	learn: 0.5750229	total: 3.45s	remaining: 16.5s
173:	learn: 0.5747074	total: 3.47s	remaining: 16.5s
174:	learn: 0.5742335	total: 3.49s	remaining: 16.5s
175:	learn: 0.5739132	total: 3.51s	remaining: 16.4s
176:	learn: 0.5735707	total: 3.53s	remaining: 16.4s
177:	learn: 0.5732115	total: 3.55s	remaining: 16.4s
178:	learn: 0.5727885	total: 3.56s	remaining: 16.3s
179:	learn: 0.5723347	total: 3.58s	remaining: 16.3s
180:	learn: 

323:	learn: 0.5190531	total: 6.62s	remaining: 13.8s
324:	learn: 0.5187164	total: 6.64s	remaining: 13.8s
325:	learn: 0.5184063	total: 6.66s	remaining: 13.8s
326:	learn: 0.5180813	total: 6.68s	remaining: 13.8s
327:	learn: 0.5176449	total: 6.7s	remaining: 13.7s
328:	learn: 0.5173322	total: 6.72s	remaining: 13.7s
329:	learn: 0.5169200	total: 6.75s	remaining: 13.7s
330:	learn: 0.5166492	total: 6.76s	remaining: 13.7s
331:	learn: 0.5163829	total: 6.79s	remaining: 13.7s
332:	learn: 0.5160939	total: 6.81s	remaining: 13.6s
333:	learn: 0.5158485	total: 6.83s	remaining: 13.6s
334:	learn: 0.5155353	total: 6.85s	remaining: 13.6s
335:	learn: 0.5152189	total: 6.87s	remaining: 13.6s
336:	learn: 0.5146833	total: 6.89s	remaining: 13.6s
337:	learn: 0.5143478	total: 6.92s	remaining: 13.5s
338:	learn: 0.5139523	total: 6.94s	remaining: 13.5s
339:	learn: 0.5135204	total: 6.96s	remaining: 13.5s
340:	learn: 0.5131290	total: 6.98s	remaining: 13.5s
341:	learn: 0.5127824	total: 7s	remaining: 13.5s
342:	learn: 0.51

483:	learn: 0.4699065	total: 9.79s	remaining: 10.4s
484:	learn: 0.4695319	total: 9.81s	remaining: 10.4s
485:	learn: 0.4693028	total: 9.84s	remaining: 10.4s
486:	learn: 0.4690337	total: 9.86s	remaining: 10.4s
487:	learn: 0.4687575	total: 9.87s	remaining: 10.4s
488:	learn: 0.4684131	total: 9.89s	remaining: 10.3s
489:	learn: 0.4680575	total: 9.91s	remaining: 10.3s
490:	learn: 0.4677205	total: 9.93s	remaining: 10.3s
491:	learn: 0.4675051	total: 9.95s	remaining: 10.3s
492:	learn: 0.4672994	total: 9.97s	remaining: 10.3s
493:	learn: 0.4670007	total: 9.99s	remaining: 10.2s
494:	learn: 0.4666613	total: 10s	remaining: 10.2s
495:	learn: 0.4663487	total: 10s	remaining: 10.2s
496:	learn: 0.4660465	total: 10.1s	remaining: 10.2s
497:	learn: 0.4658150	total: 10.1s	remaining: 10.2s
498:	learn: 0.4655036	total: 10.1s	remaining: 10.1s
499:	learn: 0.4652972	total: 10.1s	remaining: 10.1s
500:	learn: 0.4650009	total: 10.1s	remaining: 10.1s
501:	learn: 0.4647079	total: 10.1s	remaining: 10.1s
502:	learn: 0.46

644:	learn: 0.4255826	total: 13s	remaining: 7.13s
645:	learn: 0.4252606	total: 13s	remaining: 7.11s
646:	learn: 0.4250278	total: 13s	remaining: 7.09s
647:	learn: 0.4247989	total: 13s	remaining: 7.07s
648:	learn: 0.4245677	total: 13s	remaining: 7.05s
649:	learn: 0.4242686	total: 13.1s	remaining: 7.03s
650:	learn: 0.4239246	total: 13.1s	remaining: 7.01s
651:	learn: 0.4236341	total: 13.1s	remaining: 6.99s
652:	learn: 0.4234174	total: 13.1s	remaining: 6.97s
653:	learn: 0.4230827	total: 13.1s	remaining: 6.95s
654:	learn: 0.4228784	total: 13.2s	remaining: 6.93s
655:	learn: 0.4226429	total: 13.2s	remaining: 6.91s
656:	learn: 0.4224087	total: 13.2s	remaining: 6.89s
657:	learn: 0.4221267	total: 13.2s	remaining: 6.87s
658:	learn: 0.4217754	total: 13.2s	remaining: 6.85s
659:	learn: 0.4215081	total: 13.3s	remaining: 6.83s
660:	learn: 0.4212083	total: 13.3s	remaining: 6.81s
661:	learn: 0.4209732	total: 13.3s	remaining: 6.79s
662:	learn: 0.4207724	total: 13.3s	remaining: 6.77s
663:	learn: 0.4205022	

...

417:	learn: 0.3803850	total: 47.8s	remaining: 1m 6s
418:	learn: 0.3799221	total: 47.9s	remaining: 1m 6s
419:	learn: 0.3795207	total: 48s	remaining: 1m 6s
420:	learn: 0.3790635	total: 48.1s	remaining: 1m 6s
421:	learn: 0.3785171	total: 48.2s	remaining: 1m 6s
422:	learn: 0.3780063	total: 48.3s	remaining: 1m 5s
423:	learn: 0.3776405	total: 48.4s	remaining: 1m 5s
424:	learn: 0.3769856	total: 48.5s	remaining: 1m 5s
425:	learn: 0.3765126	total: 48.6s	remaining: 1m 5s
426:	learn: 0.3761255	total: 48.7s	remaining: 1m 5s
427:	learn: 0.3756683	total: 48.8s	remaining: 1m 5s
428:	learn: 0.3751524	total: 48.9s	remaining: 1m 5s
429:	learn: 0.3747837	total: 49s	remaining: 1m 4s
430:	learn: 0.3742733	total: 49.1s	remaining: 1m 4s
431:	learn: 0.3737418	total: 49.2s	remaining: 1m 4s
432:	learn: 0.3732334	total: 49.3s	remaining: 1m 4s
433:	learn: 0.3727718	total: 49.4s	remaining: 1m 4s
434:	learn: 0.3722844	total: 49.5s	remaining: 1m 4s
435:	learn: 0.3716685	total: 49.6s	remaining: 1m 4s
436:	learn: 0.37

578:	learn: 0.3135195	total: 1m 4s	remaining: 47.1s
579:	learn: 0.3131760	total: 1m 4s	remaining: 47s
580:	learn: 0.3129401	total: 1m 5s	remaining: 46.9s
581:	learn: 0.3125474	total: 1m 5s	remaining: 46.8s
582:	learn: 0.3122296	total: 1m 5s	remaining: 46.7s
583:	learn: 0.3119182	total: 1m 5s	remaining: 46.6s
584:	learn: 0.3114156	total: 1m 5s	remaining: 46.5s
585:	learn: 0.3109813	total: 1m 5s	remaining: 46.4s
586:	learn: 0.3104965	total: 1m 5s	remaining: 46.3s
587:	learn: 0.3101807	total: 1m 5s	remaining: 46.2s
588:	learn: 0.3098546	total: 1m 6s	remaining: 46.1s
589:	learn: 0.3094686	total: 1m 6s	remaining: 46s
590:	learn: 0.3090843	total: 1m 6s	remaining: 45.9s
591:	learn: 0.3088043	total: 1m 6s	remaining: 45.8s
592:	learn: 0.3084089	total: 1m 6s	remaining: 45.6s
593:	learn: 0.3081852	total: 1m 6s	remaining: 45.6s
594:	learn: 0.3078302	total: 1m 6s	remaining: 45.4s
595:	learn: 0.3075036	total: 1m 6s	remaining: 45.3s
596:	learn: 0.3071033	total: 1m 7s	remaining: 45.2s
597:	learn: 0.30

736:	learn: 0.2587732	total: 1m 21s	remaining: 29s
737:	learn: 0.2585503	total: 1m 21s	remaining: 28.9s
738:	learn: 0.2582641	total: 1m 21s	remaining: 28.8s
739:	learn: 0.2579276	total: 1m 21s	remaining: 28.6s
740:	learn: 0.2576379	total: 1m 21s	remaining: 28.5s
741:	learn: 0.2572930	total: 1m 21s	remaining: 28.4s
742:	learn: 0.2569588	total: 1m 21s	remaining: 28.3s
743:	learn: 0.2566384	total: 1m 21s	remaining: 28.2s
744:	learn: 0.2562330	total: 1m 22s	remaining: 28.1s
745:	learn: 0.2558460	total: 1m 22s	remaining: 28s
746:	learn: 0.2554766	total: 1m 22s	remaining: 27.9s
747:	learn: 0.2551393	total: 1m 22s	remaining: 27.7s
748:	learn: 0.2548426	total: 1m 22s	remaining: 27.6s
749:	learn: 0.2546165	total: 1m 22s	remaining: 27.5s
750:	learn: 0.2543901	total: 1m 22s	remaining: 27.4s
751:	learn: 0.2542049	total: 1m 22s	remaining: 27.3s
752:	learn: 0.2538519	total: 1m 22s	remaining: 27.2s
753:	learn: 0.2535853	total: 1m 22s	remaining: 27.1s
754:	learn: 0.2533528	total: 1m 23s	remaining: 27s

893:	learn: 0.2130277	total: 1m 37s	remaining: 11.5s
894:	learn: 0.2127144	total: 1m 37s	remaining: 11.4s
895:	learn: 0.2123762	total: 1m 37s	remaining: 11.3s
896:	learn: 0.2121202	total: 1m 37s	remaining: 11.2s
897:	learn: 0.2118698	total: 1m 37s	remaining: 11.1s
898:	learn: 0.2114352	total: 1m 37s	remaining: 11s
899:	learn: 0.2111600	total: 1m 37s	remaining: 10.9s
900:	learn: 0.2108344	total: 1m 38s	remaining: 10.8s
901:	learn: 0.2104277	total: 1m 38s	remaining: 10.7s
902:	learn: 0.2101213	total: 1m 38s	remaining: 10.6s
903:	learn: 0.2098683	total: 1m 38s	remaining: 10.4s
904:	learn: 0.2095498	total: 1m 38s	remaining: 10.3s
905:	learn: 0.2092117	total: 1m 38s	remaining: 10.2s
906:	learn: 0.2088061	total: 1m 38s	remaining: 10.1s
907:	learn: 0.2085308	total: 1m 38s	remaining: 10s
908:	learn: 0.2082890	total: 1m 38s	remaining: 9.9s
909:	learn: 0.2079768	total: 1m 38s	remaining: 9.79s
910:	learn: 0.2076732	total: 1m 39s	remaining: 9.68s
911:	learn: 0.2074403	total: 1m 39s	remaining: 9.57

0.987173294639807

In [64]:
from sklearn.metrics import roc_curve, precision_recall_curve
yhat = clf.predict_proba(X_test)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]
# calculate roc curves
fpr, tpr, thresholds = precision_recall_curve(y_test, yhat)
# calculate the g-mean for each threshold
gmeans = 2*fpr*tpr/(fpr+tpr)
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

Best Threshold=0.653618, G-Mean=nan


/var/folders/6r/tphhr3f17_9gplh9_dcrhzk40000gn/T/ipykernel_4308/2045578228.py:8: RuntimeWarning: invalid value encountered in true_divide
  gmeans = 2*fpr*tpr/(fpr+tpr)


In [19]:
y_pred = clf.predict(X_test)
roc_auc_score(y_test, y_pred)

0.5451475912002228

In [73]:
y_pred = yhat > 1-0.653618
f1_score(y_test, y_pred)

0.36781609195402304

In [103]:
import matplotlib.pyplot as plt
model = RandomForestClassifier(n_estimators=300,class_weight='balanced',
 criterion='gini',
 max_depth= 6,
 max_features='sqrt', random_state=42)
model.fit(X_train, y_train)
yhat = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]
# calculate roc curves
fpr, tpr, thresholds = precision_recall_curve(y_test, yhat)
# calculate the g-mean for each threshold
gmeans = 2*fpr*tpr/(fpr+tpr)
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

Best Threshold=0.547422, G-Mean=0.477


In [104]:
y_pred = yhat > 0.547422
f1_score(y_test, y_pred)

0.46783625730994155

In [106]:
final_test = pd.read_csv("test.csv")
proba = model.predict_proba(final_test.iloc[:,1:])
y_pred = proba[:,1] > 0.541203

In [107]:
final_test

id           0         1           2            3            4  \
0      0    5.351105  1.933021    2.947073 -1209.792924 -1590.254772   
1      1  567.987628  3.806155  569.333678 -1420.337690  -341.343355   
2      2 -159.797745  0.004768 -152.936798 -1549.816964  1880.071160   
3      3    7.477377  4.151975    2.659569   221.135099  1960.404537   
4      4    0.556083  1.132735    8.633191    82.622337 -1694.039840   
..   ...         ...       ...         ...          ...          ...   
795  795 -541.034705 -0.008312 -544.615414 -1592.396354  -614.480167   
796  796    3.753159  2.008486    1.454154  1089.738760  -357.088414   
797  797    9.113064 -0.216186    4.032969   446.758874   564.753214   
798  798  145.337146 -0.019249  149.552113 -1058.645300  1024.867160   
799  799    7.105814  1.927849    9.602603  1138.980072  -147.688494   

            5         6            7           8  ...          795  \
0    1.963682  0.112038   775.516167    0.424224  ...   905.747225   
1    1.996925 -0.110698     4.487823  562.999437  ...   159.436985   
2   -0.012737  1.070118     6.080241 -160.412191  ...   435.572638   
3    0.978165  2.083902     5.732444    2.915118  ...  1063.612083   
4    0.087522  1.971156     5.882287    6.228731  ...  -421.537996   
..        ...       ...          ...         ...  ...          ...   
795  1.134986  1.998570     7.803693 -539.858573  ...  -414.226747   
796  2.021897  0.090920     2.439324    2.813008  ...  1226.115548   
797 -0.094708  2.029986   888.569158    0.557762  ...   320.664389   
798  0.060729  0.190524     1.196219  151.451502  ...   813.217225   
799  0.972208  0.067037  1406.316114    9.397431  ...  1579.563667   

            796       797        798       799        800         801  \
0      4.499702  0.803534  15.112012  1.902430  66.524220    1.107305   
1    562.891324 -0.241692  15.766900 -0.058638  67.255965  562.758557   
2   -150.617012  0.003843  15.072682 -0.060233  66.337922 -158.522701   
3      7.678978  1.009009  15.884924  3.025117  67.526615    9.057854   
4      8.360745  0.079631  15.893206  0.972241  67.213155    2.137631   
..          ...       ...        ...       ...        ...         ...   
795 -548.470147  1.244007  15.878753 -0.298417  67.191183 -546.863541   
796    6.687068  0.169984  15.806151  4.122578  67.021953    4.376122   
797    1.473778  1.839615  14.320555  1.124796  65.602284    2.076574   
798  148.191448  3.721395  15.079332 -0.105967  66.601085  144.900036   
799    3.707892  0.029537  15.236567 -0.219618  66.447706   10.039949   

            802          803        804  
0      1.316054 -1343.460664  97.308018  
1    569.288638   236.871615  97.996751  
2   -160.181844   -46.134951  97.206372  
3      8.072284  -506.328267  98.280454  
4      4.197359  -188.658023  98.025160  
..          ...          ...        ...  
795 -545.404141  -216.376545  98.036042  
796    1.985985 -1434.617024  97.796732  
797    9.043637   757.438908  96.498003  
798  152.771188   611.634670  97.317049  
799    6.565464  -792.663348  97.329248  

[800 rows x 806 columns]

In [108]:
y_pred.astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [109]:
df_test = pd.DataFrame(list(zip(final_test['id'], y_pred.astype(int))), columns=["id", "target"]).set_index('id')

In [110]:
df_test

target
id         
0         0
1         0
2         0
3         0
4         0
..      ...
795       0
796       0
797       0
798       0
799       0

[800 rows x 1 columns]

In [111]:
df_test.to_csv("submission.csv")

In [114]:
clf

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7fdbd8c27fa0>,
             param_grid={'depth': [4, 6, 8],
                         'learning_rate': [0.01, 0.03, 0.05, 0.1],
                         'min_data_in_leaf': [3, 5]},
             scoring='f1')